In [34]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [35]:
import tensorflow_datasets as tfds
# tfds.disable_progress_bar()

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [36]:
get_label_name = metadata.features['label'].int2str
print(type(raw_train))

def display_img(image, label):
    plt.figure()
    plt.imshow(image, cmap=plt.cm.binary)
    plt.title(get_label_name(label))

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


In [37]:
IMG_SIZE = 160

def format_exaple(image, label):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

In [38]:
train = raw_train.map(format_exaple)
validation = raw_validation.map(format_exaple)
test = raw_test.map(format_exaple)

In [50]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

AttributeError: 'BatchDataset' object has no attribute 'shape'

In [40]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

In [41]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

In [42]:
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

In [47]:
base_learning_rate = .0001
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

/Users/yourlogarithm/.miniforge3/envs/ml/lib/python3.8/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [48]:
initial_epochs = 3
validation_steps = 20
loss0, accuracy0 = model.evaluate(validation_batches, steps=validation_steps)

2022-04-28 20:57:56.851077: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 4/20 [=====>........................] - ETA: 1s - loss: 0.9177 - accuracy: 0.3828

Corrupt JPEG data: 162 extraneous bytes before marker 0xd9


17/20 [========================>.....] - ETA: 0s - loss: 0.8782 - accuracy: 0.4375

Corrupt JPEG data: 252 extraneous bytes before marker 0xd9


20/20 [==============================] - 4s 75ms/step - loss: 0.8713 - accuracy: 0.4422


In [49]:
history = model.fit(train_batches, epochs=initial_epochs, validation_data=validation_batches)
print(history.history['accuracy'])

Epoch 1/3


2022-04-28 20:58:50.623943: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


167/582 [=======>......................] - ETA: 32s - loss: 0.4722 - accuracy: 0.7322

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


200/582 [=========>....................] - ETA: 29s - loss: 0.4328 - accuracy: 0.7630

...

547/582 [===========================>..] - ETA: 2s - loss: 0.0569 - accuracy: 0.9794

Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


582/582 [==============================] - ETA: 0s - loss: 0.0568 - accuracy: 0.9793

Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


582/582 [==============================] - 52s 89ms/step - loss: 0.0568 - accuracy: 0.9793 - val_loss: 0.0560 - val_accuracy: 0.9798
[0.8910263180732727, 0.975067138671875, 0.9793121814727783]


In [ ]:
model.save('cats_vs_dogs.h5')
new_model = tf.keras.models.load_model('cats_vs_dogs.h5')